---
## **FIRST:**
(Basic housekeeping) (2.5 minutes)
---

In [ ]:
#@title Downloading JSON key from DROPBOX.

!wget 'https://www.dropbox.com/scl/fi/pn62ld9lasx7pql9stchk/tile-bot-405312-8bb4e65cbe47.json?rlkey=izjxkv7pygdtjkj0r494z16zg&dl=0'

In [ ]:
#@title Installing/upgrading gspread libraries

!pip install gspread oauth2client
!pip install --upgrade gspread

In [ ]:
#@title Installing Selenium

# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium
pip install webdriver-manager

---
## **SECOND:**
---

In [ ]:
#@title Downloading Flags folder from Dropbox

!wget -O Flags.zip "https://www.dropbox.com/scl/fo/z2ao7680uo1avlxry68zg/h?rlkey=0yr70n4b7v3ol37jyhmbfflnh&dl=0"
!mkdir -p Flags
!unzip Flags.zip -d Flags
!rm Flags.zip

In [ ]:
#@title Doing the prepromotes

import os
import time
import zipfile
import subprocess
import shutil
from google.colab import files
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, ElementClickInterceptedException
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

folder_name = 'Prepromotes'
os.makedirs(folder_name, exist_ok=True)
download_default_directory = f'/content/{folder_name}'

#logos image input
logos_base_path = '/content/Flags'

# Google Sheets API setup
creds = Credentials.from_service_account_file('/content/tile-bot-405312-8bb4e65cbe47.json?rlkey=izjxkv7pygdtjkj0r494z16zg&dl=0')
service = build('sheets', 'v4', credentials=creds)

# Your Google Sheet ID and the range of cells to access
SPREADSHEET_ID = '1xw1M9m80rF014Uop1m9CZ-ow0w0IbYMbBbkLftsakC8' #@param {type:"string"}
RANGE_NAME = 'Tilebot!A1:M200' #@param {type:"string"}



# Set up headless Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option("prefs", {
  "download.default_directory": f'/content/{folder_name}',
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

# Call the Sheets API to read data
sheet_service = service.spreadsheets()
result = sheet_service.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values', [])

# Function to clear input field
def clear_input_field(element):
    # Clear using Selenium's clear method
    element.clear()
    # If the element is not cleared, use JavaScript to clear it
    browser.execute_script("arguments[0].value = '';", element)
    # Check and retry if the input is not cleared
    for i in range(3):
        if element.get_attribute('value'):
            element.clear()
            browser.execute_script("arguments[0].value = '';", element)
            time.sleep(1)
        else:
            break

def wait_for_download_complete(download_directory, timeout=300):
    start_time = time.time()
    while True:
        # Find the most recent zip file in the directory
        zip_files = sorted([f for f in os.listdir(download_directory) if f.endswith('.zip')],
                           key=lambda f: os.path.getctime(os.path.join(download_directory, f)),
                           reverse=True)

        if zip_files:
            latest_zip = os.path.join(download_directory, zip_files[0])
            initial_size = os.path.getsize(latest_zip)

            # Wait for a short period to check if the file is still being written to
            time.sleep(1)
            if os.path.getsize(latest_zip) == initial_size:
                # If file size hasn't changed, assume download is complete
                return latest_zip

        # Check for timeout
        if (time.time() - start_time) > timeout:
            raise Exception("Timeout: File download did not complete within the specified time.")

        time.sleep(1)  # Check every 1 second

# Create a browser instance with headless Chrome
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
browser.set_window_size(1920, 1080)

# Process and iterate over Google Sheets data
if not values:
    print('No data found.')
else:
    for row in values:
        # Ensure row has enough elements
        row += [None] * (14 - len(row))

        # Unpack the row into variables
        input1_value, firstname1_value, countries1_value, input2_value, firstname2_value, countries2_value, datetime_value, round_value, gender_value, sd_value, rank1_value, rank2_value, event_data, movida_value = row

        if input1_value is None:  # Check for end of data
            break

        browser.get('https://thelivecms.prod.streamco.cloud/tile-creator/')
            # live & upcoming

        live = WebDriverWait(browser, 2, 0.2).until(
            EC.presence_of_element_located(
                (By.XPATH,  "//*[contains(text(), 'Live & Upcoming')]")
            )
        )
        live.click();

        # home & away

        home = WebDriverWait(browser, 2, 0.2).until(
            EC.presence_of_element_located(
                (By.XPATH,  "//*[contains(text(), 'Home v Away (tennis, combat, football, etc)')]")
            )
        )
        home.click()

        # logo cards

        logo = WebDriverWait(browser, 2, 0.2).until(
            EC.presence_of_element_located(
                (By.XPATH,  "//*[contains(text(), 'Logo Cards')]")
            )
        )
        logo.click();

        # prematch logo card

        # pre = WebDriverWait(browser, 2, 0.2).until(
        #     EC.presence_of_element_located(
        #         (By.XPATH,  "//*[contains(text(), 'Logo Card Prematch')]")
        #     )
        # )
        # pre.click()

        # postmatch logo card

        post = WebDriverWait(browser, 2, 0.2).until(
            EC.presence_of_element_located(
                (By.XPATH,  "//*[contains(text(), 'Logo Card Postmatch')]")
            )
        )
        post.click()

        # select sport + competition

        select1 = WebDriverWait(browser, 2, 0.2).until(
            EC.presence_of_element_located(
                (By.ID, "select-sport-type-id")
            )
        )
        select1 = Select(select1)
        select1.select_by_visible_text('Tennis')


        select2 = WebDriverWait(browser, 2, 0.2).until(
            EC.presence_of_element_located(
                (By.ID, "select-competition-id")
            )
        )
        select2 = Select(select2)

        if sd_value == 'Singles':
            select2.select_by_visible_text('TENNIS: Australian Open Singles')
        else:
            select2.select_by_visible_text('TENNIS: Australian Open Extra Comps')

        if sd_value == 'Doubles':
            fourplayers_checkbox = WebDriverWait(browser, 2, 0.2).until(
                EC.element_to_be_clickable(
                    (By.XPATH,  "//label[text()='4 Players']/preceding-sibling::input")
                )
            )
            fourplayers_checkbox.click()

        # first name 1

        if sd_value == 'Singles':
            firstname1 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//label[contains(text(), 'Opponent #1 Small text')]/following-sibling::input[@type='text']")
                )
            )
            clear_input_field(firstname1)

            firstname1 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//label[contains(text(), 'Opponent #1 Small text')]/following-sibling::input[@type='text']")
                )
            )
            firstname1.send_keys(firstname1_value)

            # first name 2

            firstname2 = browser.find_element(By.XPATH, "//label[contains(text(), 'Opponent #2 Small text')]/following-sibling::input[@type='text']")
            clear_input_field(firstname2)
            firstname2.send_keys(firstname2_value)


        if '/' in input1_value:
            # Split the string into two parts at the slash
            name1_value, name2_value = input1_value.split('/')

            # name 1

            name1 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "input[value='home1']")
                )
            )
            clear_input_field(name1)
            time.sleep(2)

            name1 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "input[value='home1']")
                )
            )
            name1.send_keys(name1_value)

            # name 2
            name2 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "input[value='home2']")
                )
            )
            clear_input_field(name2)

            name2 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "input[value='home2']")
                )
            )
            name2.send_keys(name2_value)

            # Now name1 and name2 hold the separate names
            # You can use name1 and name2 as needed in your code
        else:
            input1 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.ID, "downshift-0-input")
                )
            )
            clear_input_field(input1)

            input1 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.ID, "downshift-0-input")
                )
            )
            input1.send_keys(input1_value)

            # team 2

            input2 = browser.find_element(By.ID,"downshift-1-input")
            clear_input_field(input2)
            input2.send_keys(input2_value)


        if '/' in input2_value:
            # Split the string into two parts at the slash
            name3_value, name4_value = input2_value.split('/')

            name3 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "input[value='away1']")
                )
            )
            clear_input_field(name3)

            name3 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "input[value='away1']")
                )
            )
            name3.send_keys(name3_value)

            name4 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "input[value='away2']")
                )
            )
            clear_input_field(name4)

            name4 = WebDriverWait(browser, 10, 0.2).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "input[value='away2']")
                )
            )
            name4.send_keys(name4_value)

        # rank 1

        if sd_value == 'Singles':
            if rank1_value:  # This will be False if rank1_value is blank or None
                rank1 = WebDriverWait(browser, 10, 0.2).until(
                    EC.element_to_be_clickable(
                        (By.XPATH, "//label[contains(text(), 'Opponent #1 Rank')]/following-sibling::input[@type='text']")
                    )
                )
                rank1.send_keys(rank1_value)

        # rank 2
        if sd_value == 'Singles':
            if rank2_value:  # This will be False if rank1_value is blank or None
                rank2 = WebDriverWait(browser, 10, 0.2).until(
                    EC.element_to_be_clickable(
                        (By.XPATH, "//label[contains(text(), 'Opponent #2 Rank')]/following-sibling::input[@type='text']")
                    )
                )
                rank2.send_keys(rank2_value)

        # rank2 = browser.find_element(By.XPATH, "//label[contains(text(), 'Opponent #2 Rank')]/following-sibling::input[@type='text']")
        # clear_input_field(rank2)
        # rank2.send_keys(rank2_value)

        if '/' in countries1_value:
            # Split the string into two parts at the slash
            country1_value, country2_value = countries1_value.split('/')

            # opponent 1A flag
            file_input_opponent1A = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH, "//label[contains(text(), 'Opponent #1A Flag')]/following-sibling::input[@type='file']")
                )
            )
            file_path_opponent1A = os.path.join(logos_base_path, f"{country1_value}.png")
            file_input_opponent1A.send_keys(file_path_opponent1A)

            # opponent 1B flag
            file_input_opponent1B = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH, "//label[contains(text(), 'Opponent #1B Flag')]/following-sibling::input[@type='file']")
                )
            )
            file_path_opponent1B = os.path.join(logos_base_path, f"{country2_value}.png")
            file_input_opponent1B.send_keys(file_path_opponent1B)


        else:
            # opponent 1 flag
            file_input_opponent1 = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH, "//label[contains(text(), 'Opponent #1 Flag')]/following-sibling::input[@type='file']")
                )
            )
            file_path_opponent1 = os.path.join(logos_base_path, f"{countries1_value}.png")
            file_input_opponent1.send_keys(file_path_opponent1)

            # opponent 2 flag
            file_input_opponent2 = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH, "//label[contains(text(), 'Opponent #2 Flag')]/following-sibling::input[@type='file']")
                )
            )
            file_path_opponent2 = os.path.join(logos_base_path, f"{countries2_value}.png")
            file_input_opponent2.send_keys(file_path_opponent2)

        if '/' in countries2_value:
            # Split the string into two parts at the slash
            country3_value, country4_value = countries2_value.split('/')

            # opponent 2A flag
            file_input_opponent2A = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH, "//label[contains(text(), 'Opponent #2A Flag')]/following-sibling::input[@type='file']")
                )
            )
            file_path_opponent2A = os.path.join(logos_base_path, f"{country3_value}.png")
            file_input_opponent2A.send_keys(file_path_opponent2A)

            # opponent 2B flag
            file_input_opponent2B = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH, "//label[contains(text(), 'Opponent #2B Flag')]/following-sibling::input[@type='file']")
                )
            )
            file_path_opponent2B = os.path.join(logos_base_path, f"{country4_value}.png")
            file_input_opponent2B.send_keys(file_path_opponent2B)


        # delete event long

        event_long = WebDriverWait(browser, 10, 0.2).until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "input[value='Event Long']")
            )
        )
        clear_input_field(event_long)

        if event_data:  # Check if event_data is not empty
            event_long.send_keys(event_data)
        else:  # If event_data is empty or None
            event_long.send_keys(" ")


        # select kick off

        # select3 = WebDriverWait(browser, 10).until(
        #     EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'Live Time Start Text')]/following-sibling::select"))
        # )

        # select_object = Select(select3)
        # select_object.select_by_visible_text('Kick Off')



        # time

        # time_input = WebDriverWait(browser, 10).until(
        #     EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='time']"))
        # )

        # # Clear the field and set the new time
        # clear_input_field(time_input)

        # desired_time = datetime_value.strftime('%H %M')
        # time_input.send_keys(desired_time)

        # date (e.g Nov 6)
        # Example string, replace with your actual date string format
        datetime_str = datetime_value  # Assuming the format is something like 'YYYY-MM-DD'

        # Convert the string to a datetime object
        datetime_obj = datetime.strptime(datetime_str, '%d/%m/%Y %H:%M')



        date_input = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable(
                (By.XPATH, "//input[@placeholder='competition date']")
            )
        )
        date_input = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='competition date']"))
        )
        clear_input_field(date_input)
        desired_date = datetime_obj.strftime('%b %-d')
        date_input.send_keys(desired_date)

        # round

        round_textarea = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//label[contains(text(), 'Round')]/following-sibling::textarea"))
        )
        clear_input_field(round_textarea)
        if sd_value == 'Doubles':
            round_textarea.send_keys(f"{gender_value} {sd_value} {round_value}")
        else:
            round_textarea.send_keys(f"{gender_value} {round_value}")

        # comp - flag

        select4 = WebDriverWait(browser, 2, 0.2).until(
            EC.presence_of_element_located(
                (By.ID, "select-special-carousel-logo-id")
            )
        )
        select4 = Select(select4)
        select4.select_by_visible_text('TENNIS: US Open Extras')

        time.sleep(2)

        # downloading

        try:
            # Wait for the button to be clickable by both class and text, with a timeout of 10 seconds
            save_as_zip_button = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn-warning') and contains(text(), 'Save as .zip')]"))
            )
            time.sleep(5)

            # Instead of clicking the button with Selenium's click function, we use JavaScript to click
            browser.execute_script("arguments[0].click();", save_as_zip_button)
            print("Button clicked successfully via JavaScript.")
        except TimeoutException:
            print("Button was not found within the given time.")
        except NoSuchElementException:
            print("Button could not be found.")
        except Exception as e:
            print(f"An error occurred: {e}")
        download_directory = download_default_directory
        try:
            downloaded_file = wait_for_download_complete(download_directory)
            print(f"Download complete: {downloaded_file}")
        except Exception as e:
            print(str(e))

        zip_files = sorted([f for f in os.listdir(download_directory) if f.endswith('.zip')], key=lambda f: os.path.getctime(os.path.join(download_directory, f)), reverse=True)

        if sd_value == 'Singles':
            if zip_files:
                zip_file_name = zip_files[0]
                zip_file_path = os.path.join(download_directory, zip_file_name)

                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(download_directory)
                    extracted_files = zip_ref.namelist()

                if extracted_files:
                    new_singles_folder_path = os.path.join(download_directory, f"{input1_value} v {input2_value} {sd_value}")
                    if not os.path.exists(new_singles_folder_path):
                        os.mkdir(new_singles_folder_path)

                    print(f"Moving files to {new_singles_folder_path}")

                    for file in extracted_files:
                        original_file_path = os.path.join(download_directory, file)
                        new_file_path = os.path.join(new_singles_folder_path, file)
                        # print(f"Moving {original_file_path} to {new_file_path}")
                        shutil.move(original_file_path, new_file_path)

                    subprocess.run(["open", "-R", new_singles_folder_path])

                else:
                    print("No files found in the zip.")
            else:
                print("No zip files found in the download directory.")

            os.remove(zip_file_path)

        else:
            name1_value, name2_value = input1_value.split('/')
            name3_value, name4_value = input2_value.split('/')
            if zip_files:
                zip_file_name = zip_files[0]
                zip_file_path = os.path.join(download_directory, zip_file_name)

                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(download_directory)
                    extracted_files = zip_ref.namelist()

                if extracted_files:
                    new_doubles_folder_path = os.path.join(download_directory, f"{name1_value} v {name3_value} {sd_value}")
                    if not os.path.exists(new_doubles_folder_path):
                        os.mkdir(new_doubles_folder_path)

                    print(f"Moving files to {new_doubles_folder_path}")

                    for file in extracted_files:
                        original_file_path = os.path.join(download_directory, file)
                        new_file_path = os.path.join(new_doubles_folder_path, file)
                        # print(f"Moving {original_file_path} to {new_file_path}")
                        shutil.move(original_file_path, new_file_path)

                    subprocess.run(["open", "-R", new_doubles_folder_path])

                else:
                    print("No files found in the zip.")
            else:
                print("No zip files found in the download directory.")

            os.remove(zip_file_path)

browser.quit()

folder_path = folder_name  # replace with your folder path
zip_file = f'{folder_name}.zip'  # name of the resulting zip file

# Compress the folder
os.system(f'zip -r {zip_file} {folder_path}')

# Download the zip file
files.download(zip_file)

---
## **MISC:**
---

In [ ]:
import shutil

dir_path = 'Prepromotes'

# Remove the directory along with all its contents
shutil.rmtree(dir_path)


In [ ]:
#@title Copying data from two Google Sheets

import gspread


def copy_sheet_data(source_sheet_id, source_worksheet_name, dest_sheet_id, dest_worksheet_name, credentials_file):
    # Authenticate with the Google Sheets API using the uploaded credentials file
    credentials = '/content/tile-bot-405312-8bb4e65cbe47.json?rlkey=izjxkv7pygdtjkj0r494z16zg&dl=0' #@param {type: "string"}
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)
    client = gspread.authorize(creds)

    # Open the source and destination Google Sheets
    try:
        source_sheet = client.open_by_key(source_sheet_id)
        dest_sheet = client.open_by_key(dest_sheet_id)
    except Exception as e:
        print(f"Error opening sheets: {e}")
        return

    # Access the specific worksheets
    try:
        source_worksheet = source_sheet.worksheet(source_worksheet_name)
        dest_worksheet = dest_sheet.worksheet(dest_worksheet_name)
    except Exception as e:
        print(f"Error accessing worksheets: {e}")
        return

    # Read data from the source worksheet
    try:
        data = source_worksheet.get_all_values()
        print(f"Data read from source: {data[:5]}")  # Print first 5 rows for debugging
    except Exception as e:
        print(f"Error reading data from source worksheet: {e}")
        return

    # Write data to the destination worksheet
    try:
        dest_worksheet.update('A1', data)
        print(f"Data written to destination worksheet successfully.")
    except Exception as e:
        print(f"Error writing data to destination worksheet: {e}")

# Example usage
source_sheet_id = '1xw1M9m80rF014Uop1m9CZ-ow0w0IbYMbBbkLftsakC8'  #@param {type:"string"}
source_worksheet_name = 'Template'  #@param {type:"string"}
dest_sheet_id = '1xw1M9m80rF014Uop1m9CZ-ow0w0IbYMbBbkLftsakC8'  #@param {type:"string"}
dest_worksheet_name = 'Day 1' #@param {type:"string"}

copy_sheet_data(source_sheet_id, source_worksheet_name, dest_sheet_id, dest_worksheet_name, credentials_file)
